## Phase-field pre-processor

This notebooks demonstrates how to use the code to:
- Associate a MatFlow DAMASK workflow with a phase-field pre-processor
- Select a slice of the volume-element for grain clustering
- Perform the grain clustering for the specified slice using MTEX, and save a grain ID map with the clusterer data
- Generate a set of seed points across the clustered grain map that are distributed according to the estimated dislocation density
- Perform a Voronoi tessellation on the seed points to generate a sub-grain map
- Generate a CIPHER input file using the sub-grain map

*Note*: this code is known to work with these software versions (but probably works with other versions too):
 - Python 3.7
 - Matlab R2020a
 - MTEX 5.7.0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px

from clustering import PhaseFieldModelPreProcessor

### Setting up the `PhaseFieldModelPreProcessor`

We generate a new `PhaseFieldModelPreProcessor` object that is associated with our MatFlow DAMASK simulation. This will generate a `segmentation` sub-directory within the workflow directory.

In [ ]:
wk_path = '/path/to/matflow/workflow'
pre_proc = PhaseFieldModelPreProcessor(workflow_dir=wk_path)

### Setting up the `Clusterer`

Each MatFlow DAMASK simulation can be associated with multiple `Clusterer` objects, which correspond to the selection of a given slice through the volume element, and with clustering parameters:

In [ ]:
# Slice selection and clustering parametrisation:
clusterer_args = {
    'element_idx': 0,
    'slice_selection': {
        'increment_idx': -1,
        'is_periodic': True,
        'x': None,
        'y': 0,
        'z': None,
        'eye': '+y',
        'up': '+z',
    },
    'method': 'MTEX-FMC', # fast-multiscale clustering, using MTEX
    'parameters': {
        'C_Maha': 2.0,
        'smoothing': 16,
    },
}

clst = pre_proc.get_clusterer(**clusterer_args)

### Running the segmentation

This may take a while, depending on the RVE resolution. At the end, we save a pickle file with the results, which can later be reloaded to avoid repeating the segmentation process (see below).

In [ ]:
clst.do_segmentation() # also does post-processing and saves results

### Loading a previous segmentation

We can load the previous segmentation results, so the `do_segmentation` does not need to be repeated:

In [ ]:
clst.load()

In [ ]:
clst.show_field_data('phase')

In [ ]:
px.imshow(clst.grain_IDs)

In [ ]:
clst.show_estimated_dislocation_density()

In [ ]:
clst.set_seed_points(
    method='dislocation_density',
    pixel_length=0.4e-6,    
    redo=True, # set seed points again, even if already set
    random_seed=413402, # use the random_seed to get the same result every time
    
) 
clst.tessellate_seed_points()
clst.save() # re-save with the additional tessellation info

In [ ]:
clst.show_tessellated_sub_grain_IDs()

### Generate CIPHER input file

Here we write the CIPHER input YAML file. Materials should be a dict whose keys correspond to the DAMASK simulation phases. Interfaces should be a dict whose keys are combinations of DAMASK simulation phases separated by a dash (like `phase1-phase2`), with a `-HAGB` or `-LAGB` suffix for the like-phase interfaces, which represet high-angle and low-angle grain boundaries, respectively. For two phases, we must specify:

- `phase1-phase2`
- `phase1-phase1-HAGB`
- `phase1-phase1-LAGB`
- `phase2-phase2-HAGB`
- `phase2-phase2-LAGB`

In `solution_parameters`, sensible values for `interfacewidth` and `maxnrefine` will be automatically set if not specified.

In [ ]:
materials = {
    "Ti-alpha": {
        "chemicalenergy": "none",
        "molarvolume": 1e-5,
        "temperature0": 500.0,
    },
    "Ti-beta": {
        "chemicalenergy": "none",
        "molarvolume": 1e-5,
        "temperature0": 500.0,
    },
}

interface_properties = {
    "Ti-alpha-Ti-beta": {
        "energy": {"e0": 0.5},
        "mobility": {"m0": 1.0e-11},
    },
    "Ti-alpha-Ti-alpha-HAGB": {
        "energy": {"e0": 0.5},
        "mobility": {"m0": 1.0e-11},
    },
    "Ti-alpha-Ti-alpha-LAGB": {
        "energy": {"e0": 0.2},
        "mobility": {"m0": 5.0e-11},
    },
    "Ti-beta-Ti-beta-HAGB": {
        "energy": {"e0": 0.5},
        "mobility": {"m0": 1.0e-11},
    },
    "Ti-beta-Ti-beta-LAGB": {
        "energy": {"e0": 0.2},
        "mobility": {"m0": 5.0e-11},
    },
}

solution_parameters = {
    "time": 1000000,
    "initblocksize": [1, 1],
    "initrefine": 8,
    "minnrefine": 0,
    "initcoarsen": 6,
    "amrinterval": 25,
    "outputfreq": 100,
    "outfile": "test_run",
    "interpolation": "cubic",
    "reltol": 1e-4,
    "abstol": 1e-4,
    "petscoptions": "-ts_adapt_monitor -ts_rk_type 2a",
    "random_seed": 1579993586,
}

inp = clst.get_cipher_input(
    materials=materials,
    interface_properties=interface_properties,
    components=["ti"],
    outputs=["phaseid", "matid", "interfaceid"],
    solution_parameters=solution_parameters,
    grid_size_power_of_two=8,
    intra_material_interface_segmented_label="HAGB",
    intra_material_interface_tessellated_label="LAGB",
)

# Write the input file:
inp.write_yaml("demo_seg_cipher.yaml")


In [ ]:
px.imshow(inp.geometry.voxel_phase)

In [ ]:
px.imshow(inp.geometry.get_interface_idx())

In [ ]:
px.imshow(inp.geometry.interface_map)

In [ ]:
inp.geometry.show()